<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPMLGymQLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning
## Appliqué à [Gym.OpenAI](https://gym.openai.com)

---
## Présentation de Gym

Voir la page d'introduction à [Gym](https://gym.openai.com).

### Installation de gym 

#### Outil AUTRE QUE COLAB (pyzo, jupyter lab, .....)

In [ ]:
!pip install gym

In [10]:
import gym

---
## Test de ML par Q-Learning pour atteindre l'objectif
- Utiliser l'environnement `FrozenLake8x8-v0` (un labyrinthe en mode texte)
- 4 actions sont possibles (Left(0), Down(1), Right(2), Up(3))
  - l'adjectif "Frozen" signifie qu'une *action n'est pas déterministe !*
    - à partir d'une case "gelée", aller à droite peut .. mener à droite, ou pas
    - => intérêt du Q-Learning adapté à ce type d'environnement probabiliste
- Le labyrinthe est ainsi composé de zones glacées, de puits, et d'un objectif

En vous basant sur les codes présent dans l'article [Q-Learning en Java](http://emmanuel.adam.free.fr/site/spip.php?article134), programmer un algo de Q-Learning pour apprendre à atteindre l'objectif 

**N.B.** 
  - *Cet environnement fonctionne bien sous colab, jupyterlab.. quelques soucis de l'affichage de l'état courant (carré rouge) sous Pyzo....* 
  - Il est fortement conseillé de débuter avec un environnement déterministe pour évaluer la bonne marche de l'algo de Q-Learning que vous aurez développer..
    - pour cela, lancer l'environnement avec 

---
#### Etude de l'environnement

In [11]:
env = gym.make('FrozenLake-v0') # tester FrozenLake8x8 pour l'environnement plus large
print(env.spec)
print(env.action_space) #ici 4 actions discrétisée
print(env.observation_space) # ici 4x4 cellules possibles

env.reset()
env.render()
print("S = Start, G = Goal, H = Hole, F = Frozen place")

EnvSpec(FrozenLake-v0)
Discrete(4)
Discrete(16)

SFFF
FHFH
FFFH
HFFG
S = Start, G = Goal, H = Hole, F = Frozen place


---
#### Test des actions

In [3]:
###### Test des fonctions
env.reset()
action = 0
observation, reward, done, info = env.step(action)
env.render()
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)

  (Left)
SFFF
FHFH
FFFH
HFFG
pos° actuelle: 0 ,gain: 0.0 ,fini: False , {'prob': 0.3333333333333333}


In [4]:
env.reset()
action = 1
observation, reward, done, info = env.step(action)
env.render()
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)

  (Down)
SFFF
FHFH
FFFH
HFFG
pos° actuelle: 1 ,gain: 0.0 ,fini: False , {'prob': 0.3333333333333333}


In [5]:
env.reset()
action = 2
observation, reward, done, info = env.step(action)
env.render()
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)

  (Right)
SFFF
FHFH
FFFH
HFFG
pos° actuelle: 0 ,gain: 0.0 ,fini: False , {'prob': 0.3333333333333333}


In [6]:
env.reset()
action = 3
observation, reward, done, info = env.step(action)
env.render()
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)

  (Up)
SFFF
FHFH
FFFH
HFFG
pos° actuelle: 1 ,gain: 0.0 ,fini: False , {'prob': 0.3333333333333333}


In [7]:
#note : 
## observation = position où se trouve l'agent
## reward = recompense
## done = but atteint
## info = proba de reussite de l'action

On est clairement ici dans un environnement non déterministe (une même action à partir d'un même état ne mène pas toujours au même résultat); c'est le contexte de prédilection de l'algo de Q-Learning...

---
### <font color="red">Mise en place de l'environnement en mode déterministe</font>
Important, pour valider l'apprentissage de votre algorithme avant de passer en mode non-déterministe, il vaut mieux le tester sur un environnement où chaque action à 100% de réussite. Ci-dessous un exemple sur le mini labyrinthe.

In [3]:
env = gym.make('FrozenLake-v0', is_slippery=False)
env.reset()
action = 2
observation, reward, done, info = env.step(action)
env.render()
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)
action = 2
observation, reward, done, info = env.step(action)
env.render()
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)
action = 1
observation, reward, done, info = env.step(action)
env.render()
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)
action = 1
observation, reward, done, info = env.step(action)
env.render()
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)
action = 1
observation, reward, done, info = env.step(action)
env.render()
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)
action = 2
observation, reward, done, info = env.step(action)
env.render()
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)


  (Right)
SFFF
FHFH
FFFH
HFFG
pos° actuelle: 1 ,gain: 0.0 ,fini: False , {'prob': 1.0}
  (Right)
SFFF
FHFH
FFFH
HFFG
pos° actuelle: 2 ,gain: 0.0 ,fini: False , {'prob': 1.0}
  (Down)
SFFF
FHFH
FFFH
HFFG
pos° actuelle: 6 ,gain: 0.0 ,fini: False , {'prob': 1.0}
  (Down)
SFFF
FHFH
FFFH
HFFG
pos° actuelle: 10 ,gain: 0.0 ,fini: False , {'prob': 1.0}
  (Down)
SFFF
FHFH
FFFH
HFFG
pos° actuelle: 14 ,gain: 0.0 ,fini: False , {'prob': 1.0}
  (Right)
SFFF
FHFH
FFFH
HFFG
pos° actuelle: 15 ,gain: 1.0 ,fini: True , {'prob': 1.0}


---
***Aide Python : (plusieurs possibilités)***
- Il n'est pas nécessaire de créer des classes Etat, ....
- Il faut pouvoir stocker les récompenses de chaque actions à partir de chaque case
  - a priori, la création d'une matrice 4x4 de 4 valeurs peut être utile.
  Plus simplement, on utilisera un tableau de n cases (pour les n etats) contenant m valeurs d'actions 
  

In [4]:
import numpy as np
# cree un tableau de n cases x m actions
q_actions = np.zeros((env.observation_space.n, env.action_space.n), np.float32)
print(q_actions)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


*Recherche du max et de sa position*

Reprenons la derniere modélisation sous forme de tableau de 4 valeurs.

In [5]:
#Exemple de recherche..
#1. ici pour l'exemple, on place des valeurs aléatoires pour les actions
Q = np.random.random((env.observation_space.n, env.action_space.n))
print(Q)

[[0.02622842 0.66972618 0.36528142 0.16282927]
 [0.35122529 0.70482077 0.01739524 0.72584099]
 [0.92954029 0.07122428 0.3912813  0.42577667]
 [0.29128913 0.79147529 0.15384777 0.3966068 ]
 [0.97758907 0.51781588 0.91098753 0.47909045]
 [0.86627767 0.54707234 0.43119216 0.16749241]
 [0.77557101 0.93507402 0.88929957 0.51822407]
 [0.7201939  0.44695228 0.23479248 0.7190885 ]
 [0.34345234 0.28705116 0.87345751 0.06094131]
 [0.47562476 0.49259948 0.74396318 0.38673388]
 [0.59985064 0.87010937 0.90973899 0.25678893]
 [0.67455614 0.0709547  0.35044194 0.25513796]
 [0.6611748  0.29230453 0.20612561 0.85008031]
 [0.88951206 0.69885898 0.88880405 0.68381282]
 [0.47375454 0.94537082 0.23895119 0.09211094]
 [0.27308147 0.54562246 0.58748761 0.60326617]]


In [6]:
#max de la case 2
max_2 = np.max(Q[2, :])
position_max_2 = np.argmax(Q[2,:])
print("case 2, max=",max_2, ", en position ", position_max_2)
print("la meilleure action de l'etat 2 serait donc ", position_max_2)

case 2, max= 0.9295402890867371 , en position  0
la meilleure action de l'etat 2 serait donc  0


In [7]:
Q[2,:]

array([0.92954029, 0.07122428, 0.3912813 , 0.42577667])

  -----
# Développer la solution en Q-Learning

Pour rappel, l'algorithme de Q-Learning est le suivant : 

- Initialiser la table des poids des arcs :
  - $\forall s \forall a Q(s,a) \gets 0$
- Pour n boucles d'apprentissage
  - $\lambda \gets 1 ; \epsilon \gets 1 ;$
  - $etat Courant \gets  etat initial$
  - Pour n' itérations ( n' = nombre max supposé d'actions pour atteindre l'objectif)
    - $s \gets  etat Courant$
    - $nb \gets random(0,1)$
    - Si $nb < \epsilon$ Alors
      - Choisir aléatoirement une action a
    - Sinon
      - Choisir action la plus intéressante : $a \gets argMax_{a'}(Q(s,a'))$
    - Fin Si
    - Effectuer action a :
    - $s' \gets  nouvel Etat Courant$
    - calculer $Q(s,a) \gets \lambda \times (r + \gamma \times max_{a'}(Q(s', a')) + (1-\lambda ) \times Q(s,a)$
    - *decrémenter* les coefficients d'apprentissage et d'exploration :
      - $\lambda \gets 0.99 \times \lambda$
      - $\epsilon \gets 0.99 \times \epsilon$
    - Si but atteint, casser la boucle n' (uniquement celle là)
  - Fin pour n' itérations
- Fin pour n boucles d'apprentissage  

En reprenant les définitions précédentes, $Q(s,a)$ est équivalent à `Q[s][a]`

## Initialisation des variables

On utilisera initialement les affectations suivantes pour les variables : 
- variables d'exploration, de récompense et d'apprentissage : 
  - $ \lambda \gets 0.8$
  - $ \gamma \gets 0.998$
- nombre de cycles : 
  - $ n \gets 10000$ si grille 4x4, 50000 si grille 8x8
  - $ n' \gets 10$ si grille 4x4, 20 si grille 8x8
    
----    

In [ ]:
import random as rnd

In [ ]:
 def q_learn(nb=12, eps = 0.99):
        q = np.zeros((env.observation_space.n, env.action_space.n), float)
        lmbda = 0.8
        gam = 0.8        
        s  = 0
        done = True
        while i<nb and not done
            x = rnd.random()
            if x<eps:
                a = rnd.randint(0, env.action_space.n)
            else:
                a = np.argmax(Q[s,:])
            sprim, r, done, _ = env.step(action)
            Q[s,a] = lmbda * (r + gam * np.max(Q[sprim,:])) + (1-lmda) * Q[s,a]
            lmbda = lmbda * 0.99   
            i = i + 1
        eps = eps * 0.99

        


In [9]:
observation, _, done, _ = env.step(action)

NameError: name 'env' is not defined

In [8]:
rnd.randint(0,9)

6

-----

### Double Q Learning

L'algo de **double** Q Learning simple converge plus rapidement; il repose sur ces équations :

$a^* \gets  argmax_{a} QA(s', a)$

$QA(s,a) \gets QA(s,a) + \lambda \times (r + \gamma \times QB(s', a^*))-  QA(s,a))$ 

et

$b^* \gets  argmax_{a} QB(s', a)$

$QB(s,a) \gets QB(s,a) + \lambda \times (r + \gamma \times QA(s', b^*))-  QB(s,a))$ 

avec
  - $\lambda$ : coef d'apprentissage
  - $\gamma$ : coef de réduction 
  - $r$ : récompense
  
----
- Coder cet algorithme et vérifier la convergence plus rapide.
- Selon-vous pourquoi est-ce-que le double Q-Learning et plus rapide que le Q-Learning ?